In [10]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

In [11]:
url ='https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&sop=and&sst=subscriber_cnt&sod=desc&sfl=&stx=&sca=&page=1'
res = requests.get(url)
soup = BeautifulSoup(res.text)
trs = soup.select('#list-skin > form:nth-child(4) > table > tbody > tr')

In [12]:
trs = soup.select('#list-skin > form:nth-child(4) > table > tbody > tr')
len(trs)

100

In [13]:
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver.exe')
driver.get(url)

In [14]:
soup = BeautifulSoup(driver.page_source)
trs = soup.select('.aos-init')
tr = trs[0]

In [15]:
def convert(s):
    s = s.replace('억','').replace('개','').replace(',','').replace('만','0000')
    return int(s)

In [16]:
import time
lines = []
for page in range(1,11):
    url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&sop=and&sst=subscriber_cnt&sod=desc&sfl=&stx=&sca=&page=' + str(page)
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    trs = soup.select('.aos-init')
    for tr in trs:
        rank = int(tr.select_one('.rank').get_text().strip())
        category = tr.select_one('.category').get_text().strip()[1:-1]
        channel = tr.select_one('.subject > h1 > a').get_text().strip()
        subscriber_cnt = convert(tr.select_one('.subscriber_cnt').get_text().strip())
        view_cnt = convert(tr.select_one('.view_cnt').get_text().strip())
        video_cnt = convert(tr.select_one('.video_cnt').get_text().strip())
        lines.append({
            '순위': rank, '카테고리':category, '채널명': channel, '구독자수': subscriber_cnt, '조회수': view_cnt,
            '비디오': video_cnt
        })
driver.close()

In [17]:
df = pd.DataFrame(lines)
df.shape

(1000, 6)

In [18]:
df.tail()

,순위,카테고리,채널명,구독자수,조회수,비디오
995,996,취미/라이프,DanoTV,730000,158420000,642
996,997,키즈/어린이,코코몽 COCOMONG TV - Cartoon & Song For Kids,730000,951630000,1420
997,998,음악/댄스/가수,Yuni Marimba 유니 마림바,730000,105130000,147
998,1001,미분류,김지윤의 지식Play,730000,59060000,186
999,1000,미분류,성창경TV,730000,1036110000,16348
